# Batch manager

In [1]:
from mesmerize_core import *
import numpy as np
from copy import deepcopy
import pandas as pd
from pathlib import Path
import os

2023-02-13 10:32:31.467719: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Paths and infos

In [2]:
#set_parent_raw_data_path("/Users/davide/caiman_data/")
set_parent_raw_data_path('/ceph/imaging1/arie')
batch_path = Path('/ceph/imaging1/arie/prepocessing/429420_toms/batch_toms.pickle')

#dict with animal name and number
animal_dict = {'toms': 429420,'crocs':429419}


# Load batch

In [3]:
df = load_batch(batch_path)
df.head()

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,toms_429420_20230202,429420_toms/20230202_429420/20230202_429420_00...,"{'main': {'max_shifts': (48, 48), 'strides': (...","{'success': False, 'traceback': 'multiprocessi...",2023-02-11T17:00:34,2023-02-11T17:37:51,2215.52 sec,None,aad4e4e4-7ea9-4ee8-b41d-031ff9f7be4c


# Add motion correction parameters
Add items to the batch by inputting the video path, the item name (= session name) and the parameter set for the motion correction. 
Run this cell multiple times to add multple rows with different items or parameters.

Use:

```
item_id = df.iloc[i].uuid
df.caiman.remove_item(item_id)
```

to remove the item at row i from the batch df.



In [10]:
# put movie path and session name (DATE_ANIMAL)

date = 20230202
animal_name = 'toms'
animal_num = animal_dict[animal_name]

item_name = f'{animal_name}_{animal_num}_{date}'
input_movie_path = '429420_toms/20230202_429420/20230202_429420_00001.tif'


In [11]:
# define one parameter set
mcorr_params =\
{
  'main': # this key is necessary for specifying that these are the "main" params for the algorithm
    {
        'max_shifts': [48, 48],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 10,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

In [133]:
# add an item to the batch
df.caiman.add_item(
    algo='mcorr',
    input_movie_path=input_movie_path,
    params=mcorr_params,
    item_name=item_name,  # filename of the movie, but can be anything
)

In [134]:
#visualize added items
df.tail(1)

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,toms_429420_20230202,429420_toms/20230202_429420/20230202_429420_00...,"{'main': {'max_shifts': (48, 48), 'strides': (...",None,2023-02-11T17:00:34,None,None,None,aad4e4e4-7ea9-4ee8-b41d-031ff9f7be4c


# Run given subset

In [12]:
subset = df[(df['algo']=='mcorr') & (df['item_name']==item_name)]

subset

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,toms_429420_20230202,429420_toms/20230202_429420/20230202_429420_00...,"{'main': {'max_shifts': (48, 48), 'strides': (...","{'success': False, 'traceback': 'multiprocessi...",2023-02-11T17:00:34,2023-02-11T17:37:51,2215.52 sec,None,aad4e4e4-7ea9-4ee8-b41d-031ff9f7be4c


In [13]:
# number of cpu processes:

os.environ["MESMERIZE_N_PROCESSES"] = '10'

In [14]:
# run all items in subset 
for i in range(len(subset)):
    row = subset.iloc[i]
    print(f"RUNNING {row.algo} ON ITEM: {row.item_name}")
    process = row.caiman.run()

print('DONE')

RUNNING mcorr ON ITEM: toms_429420_20230202


2023-02-13 10:33:46.354254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


starting mc


mc failed, stored traceback in output
DONE


In [15]:
df = df.caiman.reload_from_disk()
print('RUNNING OUTCOMES:')
for i in range(len(df)):
    row = df.iloc[i]
    print(f'{row["item_name"]}:{row["outputs"]["success"]}')


RUNNING OUTCOMES:
toms_429420_20230202:False


In [17]:
df.iloc[0]['outputs']['traceback']

'Traceback (most recent call last):\n  File "/memdyn/davide/mambaforge/envs/mescore/lib/python3.10/site-packages/numpy/core/memmap.py", line 255, in __new__\n    fid.flush()\nOSError: [Errno 122] Disk quota exceeded\n\nDuring handling of the above exception, another exception occurred:\n\nTraceback (most recent call last):\n  File "/memdyn/davide/mambaforge/envs/mescore/lib/python3.10/site-packages/mesmerize_core/algorithms/mcorr.py", line 67, in run_algo\n    mc.motion_correct(save_movie=True)\n  File "/memdyn/davide/mambaforge/envs/mescore/lib/python3.10/site-packages/caiman/motion_correction.py", line 259, in motion_correct\n    self.motion_correct_pwrigid(template=template, save_movie=save_movie)\n  File "/memdyn/davide/mambaforge/envs/mescore/lib/python3.10/site-packages/caiman/motion_correction.py", line 373, in motion_correct_pwrigid\n    _x_shifts_els, _y_shifts_els, _z_shifts_els, _coord_shifts_els = motion_correct_batch_pwrigid(\n  File "/memdyn/davide/mambaforge/envs/mescore

# Parameter selection
 Use the `visualize.ipynb` to plot, look at and evaluate the best set of parameters.
 Chosen parameters will be flagged with `keep=True`. The next line deletes all non chosen parameters, before proceeding to source separation


# Source separation

In [139]:
df = df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,toms_429420_20230202,429420_toms/20230202_429420/20230202_429420_00...,"{'main': {'max_shifts': (48, 48), 'strides': (...","{'success': False, 'traceback': 'multiprocessi...",2023-02-11T17:00:34,2023-02-11T17:37:51,2215.52 sec,None,aad4e4e4-7ea9-4ee8-b41d-031ff9f7be4c


In [4]:
input_row = 0

In [5]:
# some params for CNMF
params_cnmf =\
{
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'fr': 30, # framerate of the video, very important!
            'p': 1,   # order of autoregressive process contstraint
            'nb': 2,  # number of backround components       
            'merge_thr': 0.85, # correlation th to merge to sources
            'rf': 15,    # half-size of patch in pixels
            'stride': 6, # "overlap between patches in pixels, should be roughly neuron diameter
            'K': 6,      # number of neurons per pathc
            'gSig': [6, 6], # half-size of neuron in pixels (row,columns)
            'ssub': 1, # spatial compression, if larger than one compresses
            'tsub': 1, # temporal compression, if larger than one compresses
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,  # min snr for good components
            'rval_thr': 0.7, # spatial footprint consistency
            'use_cnn': False,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

In [6]:
# add a batch item
df.caiman.add_item(
    algo='cnmf', # algo is cnmf
    input_movie_path=df.iloc[input_row],  # use mcorr output from a completed batch item
    params=params_cnmf,
    item_name=df.iloc[input_row]["item_name"], # use the same item name
)

BatchItemUnsuccessfulError: Batch item was unsuccessful, traceback from subprocess:
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/memdyn/davide/mambaforge/envs/mescore/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/memdyn/davide/mambaforge/envs/mescore/lib/python3.10/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/memdyn/davide/mambaforge/envs/mescore/lib/python3.10/site-packages/caiman/motion_correction.py", line 3133, in tile_and_correct_wrapper
    outv[:, idxs] = np.reshape(
  File "<__array_function__ internals>", line 200, in reshape
  File "/memdyn/davide/mambaforge/envs/mescore/lib/python3.10/site-packages/numpy/core/fromnumeric.py", line 298, in reshape
    return _wrapfunc(a, 'reshape', newshape, order=order)
  File "/memdyn/davide/mambaforge/envs/mescore/lib/python3.10/site-packages/numpy/core/fromnumeric.py", line 57, in _wrapfunc
    r

In [116]:
df.tail(1)

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
5,cnmf,toms_429420_20230202,f9fbfa16-3ff6-458d-9163-f0de100f82e3/f9fbfa16-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",None,2023-02-11T12:02:24,None,None,None,55079dfe-3e82-4ff6-b202-5a9795415f0c


# Run source separation

In [117]:
subset = df[(df['algo']=='cnmf') & (df['item_name']==item_name)]
subset

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
3,cnmf,toms_429420_20230202,f9fbfa16-3ff6-458d-9163-f0de100f82e3/f9fbfa16-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",None,2023-02-11T12:00:48,None,None,None,d13856c9-e02d-4933-b41a-7614d5732e0d
4,cnmf,toms_429420_20230202,f9fbfa16-3ff6-458d-9163-f0de100f82e3/f9fbfa16-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",None,2023-02-11T12:00:58,None,None,None,df5db209-3163-4f33-b874-439e2ea4c085
5,cnmf,toms_429420_20230202,f9fbfa16-3ff6-458d-9163-f0de100f82e3/f9fbfa16-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",None,2023-02-11T12:02:24,None,None,None,55079dfe-3e82-4ff6-b202-5a9795415f0c


In [118]:
os.environ["MESMERIZE_N_PROCESSES"] = '10'

In [ ]:
# run all items in subset 
for i in range(len(subset)):
    row = subset.iloc[i]
    print(f"RUNNING {row.algo} ON ITEM: {row.item_name}")
    process = row.caiman.run()

print('DONE')

RUNNING cnmf ON ITEM: toms_429420_20230202


2023-02-11 12:02:44.430026: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                        toms_429420_20230202
input_movie_path    f9fbfa16-3ff6-458d-9163-f0de100f82e3/f9fbfa16-...
params              {'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...
outputs                                                          None
added_time                                        2023-02-11T12:00:48
ran_time                                                         None
algo_duration                                                    None
comments                                                         None
uuid                             d13856c9-e02d-4933-b41a-7614d5732e0d
Name: 3, dtype: object
With params:{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 'tsub': 1, 'method_init': 'greedy_roi', 'min_SNR': 2.0, '

In [122]:
df = df.caiman.reload_from_disk()

EOFError: Ran out of input

# Save final version
Deletes discarded parameters setting, saves preprocessed data to preprocessed folder destination in consistent format